# Use Case - Granule Search (FedEO)

This chapter provides a comprehensive and detailed process about how to implement a CEOS
OpenSearch client, which includes how to retrieve the IDN collection ID for the collection of
interest, and how to build an OpenSearch request.

In [1]:
import re
import json, requests, xml
import pandas as pd
# import ipywidgets as widgets

from xml.dom import minidom
# from IPython.display import Image
from xml.etree import ElementTree
# from IPython.display import HTML
from IPython.display import Markdown as md

In [2]:
def get_api_request(template, os_querystring):
  # Fill (URL) template with OpenSearch parameter values provided in os_querystring and return as short HTTP URL without empty parameters.
  
  # print("URL template: " + template)
  
  # Limitation: the OSDD may use a default namespace for OpenSearch instead of using "os".
  # We make a simple correction here allowing to use OpenSearch queryables without namespace in requests.
  # A more generic solution to obtain namespaces from the OSDD and compare them with user supplied namespaces is future work.
  
  OS_NAMESPACE = 'os:'
      
  # perform substitutions in template
  for p in os_querystring:
      # print("  .. replacing:", p, "by", os_querystring[p])
      # template = re.sub('\{'+p+'.*?\}', os_querystring[p] , template)
      result = re.subn('\{'+p+'.*?\}', os_querystring[p] , template)
      n = result[1]
      template = result[0]
      if (n<1):
          if (':' in p):
                print("ERROR: parameter " + p + " not found in template.")
          else:
                # try with explicit namespace
                result = re.subn('\{'+OS_NAMESPACE+p+'.*?\}', os_querystring[p] , template)
                n = result[1]
                template = result[0]
                if (n<1):
                    print("ERROR: parameter " + OS_NAMESPACE+p + " not found in template.")   
      
      # print("- intermediate new template:" + template)
      
  # remove empty search parameters
  template=re.sub('&?[a-zA-Z]*=\{.*?\}', '' , template)
  
  # remove remaining empty search parameters which did not have an HTTP query parameter attached (e.g. /{time:end}).
  template=re.sub('.?\{.*?\}', '' , template)
  
  # print("API request: " + template)
            
  return (template)

## FedEO Systems

There are two operational FedEO systems to which end-users have access.

- FedEO PROD – this is FedEO production instance and is available to all users.  Location: https://fedeo.ceos.org/opensearch/description.xml

- FedEO TEST – this is FedEO testing instance used by data partners and external clients to perform testing before changes are made to the FedEO production instance. Location: https://geo.spacebel.be/opensearch/description.xml

The production instance will provide access to collections verified to be
at least minimally compliant.  The collections registered in IDN are a subset of these.  The TEST instance may provide access to additional collections (e.g.,
new collections undergoing testing and not yet registered in the IDN), and capabilities which have not yet been tested sufficiently to move to the production system.

## Retrieve the Collection OSDD via IDN OpenSearch

The steps in this section are identical to the steps explained in section 6.2. To fit the end-to-end
FedEO use case, the parameter values for the collection search has been selected in such a way
that the granule search is eventually performed through FedEO.

**Step 1**  
>  Obtain the IDN OpenSearch OSDD to formulate a valid IDN OpenSearch request.

In [3]:
url_osdd = "https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc"

In [4]:
md("The OpenSearch Description Document is accessible at the fixed location [{url}]({url}) and contains the URL template to be used for collection search.".format(url=url_osdd))

The OpenSearch Description Document is accessible at the fixed location [https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc](https://cmr.earthdata.nasa.gov/opensearch/collections/descriptor_document.xml?clientId=ceosOpenSearchDoc) and contains the URL template to be used for collection search.

In [5]:
response = requests.get( url_osdd )

xmlstr = minidom.parseString(response.text).toprettyxml(indent='  ',newl='')
# md("```xml\n" + xmlstr + "\n```\n")

The OSDD may contain URL templates for multiple values for `type` (media types) and multiple values for `rel` (relations).  The `rel` values have the following meaning (if present) as defined in [[RD1]](#RD1). :

| **rel** |  **description**  | 
| ----- | ----- |
| collection |  URL template to be used for collection search.  |
| results |  URL template to be used for granule search (default).  |
| service |  URL template to be used for service or application search.  |

Find the URL template for collection search with Atom responses.

In [6]:
# find URL template for collection search
from xml.etree import ElementTree
root = ElementTree.fromstring(response.text)

ns = {'os': 'http://a9.com/-/spec/opensearch/1.1/'}
collection_url_atom = root.find('os:Url[@rel="collection"][@type="application/atom+xml"]', ns)

collection_template = collection_url_atom.attrib['template']
collection_template

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword={os:searchTerms?}&instrument={echo:instrument?}&satellite={eo:platform?}&boundingBox={geo:box?}&lat={geo:lat?}&lon={geo:lon?}&radius={geo:radius?}&geometry={geo:geometry?}&placeName={geo:name?}&startTime={time:start?}&endTime={time:end?}&cursor={os:startPage?}&numberOfResults={os:count?}&offset={os:startIndex?}&uid={geo:uid?}&hasGranules={echo:hasGranules?}&isCwic={echo:isCwic?}&isGeoss={echo:isGeoss?}&isCeos={echo:isCeos?}&isEosdis={echo:isEosdis?}&isFedeo={echo:isFedeo?}&provider={echo:provider?}&clientId=ceosOpenSearchDoc'

**Step 2**  
>  Search collections of interest through IDN OpenSearch with proper request parameters (e.g. spatial footprint, temporal extent and keyword). 

In [7]:
request_url = get_api_request(collection_template, {'count': '10', 'searchTerms': 'C1532648141-ESA'})
request_url

'https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=C1532648141-ESA&numberOfResults=10&clientId=ceosOpenSearchDoc'

In [8]:
response = requests.get( request_url )
xmlstr = minidom.parseString(response.text).toprettyxml(indent='   ', newl='')
md("```xml\n" + xmlstr + "\n```\n")

```xml
<?xml version="1.0" ?><feed esipdiscovery:version="1.2" xmlns="http://www.w3.org/2005/Atom" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom" xmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/" xmlns:esipdiscovery="http://commons.esipfed.org/ns/discovery/1.2/" xmlns:georss="http://www.georss.org/georss" xmlns:gml="http://www.opengis.net/gml" xmlns:os="http://a9.com/-/spec/opensearch/1.1/" xmlns:relevance="http://a9.com/-/opensearch/extensions/relevance/1.0/" xmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/">   
     <updated>2024-12-05T10:16:27.016Z</updated>   
     <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom</id>   
     <author>      
          <name>CMR</name>      
          <email>echodev@echo.nasa.gov</email>      
     </author>   
     <title type="text">ECHO dataset metadata</title>   
     <subtitle type="text">Search parameters: keyword =&gt; C1532648141-ESA</subtitle>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml" hreflang="en-US" rel="search" type="application/opensearchdescription+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=C1532648141-ESA&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc" hreflang="en-US" rel="self" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=C1532648141-ESA&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=1" hreflang="en-US" rel="last" type="application/atom+xml"/>   
     <link href="https://cmr.earthdata.nasa.gov/opensearch/collections.atom?keyword=C1532648141-ESA&amp;numberOfResults=10&amp;clientId=ceosOpenSearchDoc&amp;cursor=1" hreflang="en-US" rel="first" type="application/atom+xml"/>   
     <link href="https://wiki.earthdata.nasa.gov/display/echo/Open+Search+API+release+information" hreflang="en-US" rel="describedBy" title="Release Notes" type="text/html"/>   
     <os:Query os:searchTerms="C1532648141-ESA" role="request" xmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/"/>   
     <os:totalResults>1</os:totalResults>   
     <os:itemsPerPage>10</os:itemsPerPage>   
     <os:startIndex>1</os:startIndex>   
     <entry>      
          <id>https://cmr.earthdata.nasa.gov/opensearch/collections.atom?uid=C1532648141-ESA</id>      
          <author>         
               <name>CMR</name>         
               <email>echodev@echo.nasa.gov</email>         
          </author>      
          <consortium>CEOS</consortium>      
          <consortium>FEDEO</consortium>      
          <title type="text">CryoSat products</title>      
          <summary type="text">CryoSat's primary payload is the SAR/Interferometric Radar Altimeter (SIRAL) (https://earth.esa.int/eogateway/instruments/siral) which has extended capabilities to meet the measurement requirements for ice-sheet elevation and sea-ice freeboard.  CryoSat also carries three star trackers for measuring the orientation of the baseline. In addition, a radio receiver called Doppler Orbit and Radio Positioning Integration by Satellite (DORIS) and a small laser retroreflector ensures that CryoSat's position will be accurately tracked.  More detailed information on CryoSat instruments is available on the CryoSat mission page.  The following CryoSat datasets are available and distributed to registered users:  Level 1B and L2 Ice products: FDM, LRM, SAR and SARIn Consolidated Level 2 (GDR): (LRM+SAR+SARIN) consolidated ice products over an orbit Intermediate Level 2 Ice products: LRM, SAR and SARIn L1b and L2 Ocean Products: GOP and IOP CryoTEMPO EOLIS Point Products CryoTEMPO EOLIS Gridded Products  Detailed information concerning each of the above datasets is available in the CryoSat Products Overview (https://earth.esa.int/eogateway/missions/cryosat/products) and in the news item: CryoSat Ocean Products now open to scientific community (https://earth.esa.int/web/guest/missions/esa-operational-eo-missions/cryosat/news/-/asset_publisher/47bD/content/cryosat-ocean-products-now-open-to-scientific-community).  CryoSat Level 1B altimetric products contain time and geo-location information as well as SIRAL measurements in engineering units. Calibration corrections are included and have been applied to the window delay computations. In Offline products, geophysical corrections are computed from Analysis Auxiliary Data Files (ADFs), whereas in FDM products corrections are computed for Forecast ADFs. All corrections are included in the data products and therefore the range can be calculated by taking into account the surface type.  The Offline Level 2 LRM, SAR and SARIn ice altimetric products are generated 30 days after data acquisition and are principally dedicated to glaciologists working on sea-ice and land-ice areas. The Level 2 FDM products are near-real time ocean products, generated 2-3 hours after data acquisition, and fulfill the needs of some ocean operational services. Level 2 products contain the time of measurement, the geo-location and the height of the surface.  IOP and GOP are outputs of the CryoSat Ocean Processor. These products are dedicated to the study of ocean surfaces, and provided specifically for the needs of the oceanographic community. IOP are generated 2-3 days after data sensing acquisition and use the DORIS Preliminary Orbit. GOP are typically generated 30 days after data sensing acquisition and use the DORIS Precise Orbit. Geophysical corrections are computed from the Analysis ADFs, however following the oceanographic convention the corrections are available but not directly applied to the range (as for FDM).  The CryoSat ThEMatic PrOducts (Cryo-TEMPO) projects aim to deliver a new  paradigm of simplified, harmonized, and agile CryoSat-2 products, that are  easily accessible to new communities of non-altimeter experts and end users. The Cryo-TEMPO datasets include dedicated products over five thematic areas, covering Sea Ice, Land Ice, Polar Ocean, Coastal Ocean and Inland Water, together with a novel SWATH product (CryoTEMPO-EOLIS) that exploits CryoSat's SARIn mode over ice sheet margins. The standard Cryo-TEMPO products include fully-traceable uncertainties and use rapidly evolving, state-of-the-art processing dedicated to each thematic area. Throughout the project, the  products will be constantly evolved, and validated by a group of Thematic Users, thus ensuring optimal relevance and impact for the intended target  communities. More information on the Cryo-TEMPO products can be found on the Project Website (http://cryosat.mssl.ucl.ac.uk/tempo/index.html). The CryoTEMPO-EOLIS swath product exploits CryoSat's SARIn mode and the novel Swath processing technique to deliver increased spatial and temporal coverage of time-dependent elevation over land ice, a critical metric for tracking ice mass trends in support to a wide variety of end-users. The CryoTEMPO-EOLIS swath product exploits CryoSat's SARIn mode and the novel Swath processing technique to deliver increased spatial and temporal coverage of time-dependent elevation over land ice, a critical metric for tracking ice mass trends in support to a wide variety of end-users.The dataset consists of systematic reprocessing of the entire CryoSat archive to generate new L2-Swath products, increasing data sampling by 1 to 2 orders of magnitude compared with the operational L2 ESA product. In addition, the EOLIS dataset is joined with the ESA L2 Point-Of-Closest-Approach to generate monthly DEM (Digital Elevation Model) products. This dataset will further the ability of the community to analyse and understand trends across the Greenland Ice Sheet margin, Antarctica and several mountain glaciers and ice caps around the world.</summary>      
          <updated/>      
          <link href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=CryoSat.products" hreflang="en-US" length="0.0KB" rel="http://esipfed.org/ns/fedsearch/1.1/search#"/>      
          <link href="https://fedeo-client.ceos.org?parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=CryoSat.products" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://fedeo.ceos.org/opensearch/request/?httpAccept=application/vnd.iso.19139-2%2Bxml&amp;parentIdentifier=EOP:ESA:EARTH-ONLINE&amp;uid=CryoSat.products&amp;recordSchema=iso19139-2" hreflang="en-US" length="0.0KB" rel="enclosure"/>      
          <link href="https://doi.org/10.5270/CR2-120cf4c" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-cbow23i" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-05fbc11" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-41ad749" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-2cnblvi" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-fbae3cd" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-u3805kw" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-6afef01" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-k1o4pyh" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-614f341" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-53hztdl" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-388fb81" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-gsyvnx0" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-65cff05" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-he6wfkr" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-b0fffc1" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-2xs4q4l" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-11aea35" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-656eeb1" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-7ece675" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-3205d1e" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-e2dd631" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-1d3c78c" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-52fafc4" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-3f3c491" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-996c491" hreflang="en-US" rel="via"/>      
          <link href="https://doi.org/10.5270/CR2-b39c015" hreflang="en-US" rel="via"/>      
          <link href="http://gcmd.nasa.gov/getdif.htm?CryoSat.products_NA" hreflang="en-US" rel="enclosure" title="CryoSat.products" type="text/html"/>      
          <link href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=CryoSat.products" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml"/>      
          <link href="https://cmr.earthdata.nasa.gov/search/concepts/C1532648141-ESA.xml" hreflang="en-US" rel="via" title="Product metadata" type="application/xml"/>      
          <dc:identifier>C1532648141-ESA</dc:identifier>      
          <dc:date>2010-04-08T00:00:00.000Z/</dc:date>      
          <echo:datasetId>CryoSat products</echo:datasetId>      
          <echo:shortName>CryoSat.products</echo:shortName>      
          <echo:versionId>NA</echo:versionId>      
          <echo:entryId>CryoSat.products_NA</echo:entryId>      
          <echo:dataCenter>ESA</echo:dataCenter>      
          <echo:organization>ESA/ESRIN</echo:organization>      
          <echo:processingLevelId>NA</echo:processingLevelId>      
          <echo:coordinateSystem>CARTESIAN</echo:coordinateSystem>      
          <echo:orbitParameters/>      
          <georss:box>-90 -180 90 180</georss:box>      
          <echo:hasVariables>false</echo:hasVariables>      
          <echo:hasFormats>false</echo:hasFormats>      
          <echo:hasTransforms>false</echo:hasTransforms>      
          <echo:hasCombine>false</echo:hasCombine>      
          <echo:hasSpatialSubsetting>false</echo:hasSpatialSubsetting>      
          <echo:hasTemporalSubsetting>false</echo:hasTemporalSubsetting>      
          <echo:cloudHosted>false</echo:cloudHosted>      
          <relevance:score>0.5</relevance:score>      
          <echo:tag>         
               <echo:tagKey>int.esa.fedeo</echo:tagKey>         
          </echo:tag>      
          <echo:is_fedeo>true</echo:is_fedeo>      
     </entry>   
</feed>
```


**Step 3**  
>  From the IDN OpenSearch response, obtain the OSDD endpoint for the collection by parsing the href attribute under `<link rel="search" type=”application/opensearchdescription+xml” >` Note that this collection happens to be accessed via FedEO. 

In [9]:
root = ElementTree.fromstring(response.text)
# Extract <link> element with the OSDD for the granule search
el = root.find('{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}link[@rel="search"][@type="application/opensearchdescription+xml"]')
xmltxt = ElementTree.tostring(el, encoding='unicode', method='xml')
md("```xml\n" + xmltxt + "\n```\n")

```xml
<ns0:link xmlns:ns0="http://www.w3.org/2005/Atom" href="https://fedeo.ceos.org/opensearch/description.xml?parentIdentifier=CryoSat.products" hreflang="en-US" rel="search" title="Non-CMR OpenSearch Provider Granule Open Search Descriptor Document" type="application/opensearchdescription+xml" />
    
```


## Retrieve Granules via OpenSearch

After retrieving the OSDD endpoint from the IDN by querying through the CEOS OpenSearch API, OpenSearch clients will sequentially interact with the host remote server for inventory search.
The following table shows the basic information about the use case of interacting with FedEO via OpenSearch.